## Final Project - Student Predictive Model

#### Imports

In [32]:
import csv
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import scipy as sp
import random
import pandas as pd
from math import sqrt
from sklearn import datasets
from sklearn.svm import SVC 
from sklearn.linear_model import LinearRegression
import seaborn as sb

In [9]:
#Reading in Data
pre_processed_data = pd.read_csv('StudentsPerformance.csv', delimiter=',')
pre_processed_data.head(5)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


#### Helper Functions

In [38]:
def simple_plot(name, x_lable, x,y_lable, y):
    plt.scatter(x,y)
    plt.title(name)
    plt.xlabel(x_lable)
    plt.ylabel(y_lable)
    plt.show()
    
# Categorical to Numerical - Requires correct order - FIX  THIS
def convert_to_numerical(arr):
    return [
        1,
        G_map[arr[0]],
        RE_map[arr[1]],
        PE_map[arr[2]],
        L_map[arr[3]],
        TP_map[arr[4]]
    ]
    

#### Category Mapping

In [30]:
# Gender Mapping
G_map = {
    'female' : 1,
    'male' : 0
}

# Race/Ethnicity Mapping
RE_map = {
    'group A' : 0,
    'group B' : 1,
    'group C' : 2,
    'group D' : 3,
    'group E' : 4
}

# Parental Education Mapping
PE_map = {
    'some college' :        0,
    'associate\'s degree' : 1,
    'high school' :         2,
    'some high school' :    3,
    'bachelor\'s degree' :  4,
    'master\'s degree' :    5
}

# Lunch Mapping
L_map = {
    'standard' : 0,
    'free/reduced' : 1
}

# Test Prep Mapping

TP_map = {
    'none' : 0,
    'completed' : 1
}

#### Visualization of Data

In [45]:
#Visualizing Data

#Pre-Processed Individual Data
math = np.array(pre_processed_data["math score"])
reading = np.array(pre_processed_data["reading score"])
writing = np.array(pre_processed_data['writing score'])
gender = np.array(pre_processed_data['gender'])
race_ethnicity = np.array(pre_processed_data['race/ethnicity'])
parent_education = np.array(pre_processed_data['parental level of education'])
lunch = np.array(pre_processed_data['lunch'])
test_prep = np.array(pre_processed_data['test preparation course'])

# Mapped Individual Data

mapped_gender = np.array([G_map[value] for value in gender])
mapped_race_ethnicity = np.array([RE_map[value] for value in race_ethnicity])
mapped_parent_education = np.array([PE_map[value] for value in parent_education])
mapped_lunch = np.array([L_map[value] for value in lunch])
mapped_test_prep = np.array([TP_map[value] for value in test_prep])

# Processed 
average_score = (math + reading + writing) / 3
intersect = np.ones(len(average_score))

# Mapped Data Matrix
mapped_data_matrix = np.column_stack((intersect, mapped_gender, 
                               mapped_race_ethnicity, 
                               mapped_parent_education, 
                               mapped_lunch, 
                               mapped_test_prep))



print(mapped_data_matrix)
reg = LinearRegression().fit(mapped_data_matrix, average_score)
converted1 = convert_to_numerical(['female', 'group B', 'bachelor\'s degree', 'free/reduced', 'none'])
print(converted1)
print(reg.predict(np.array([converted1])))

converted2 = convert_to_numerical(['male', 'group A', 'associate\'s degree', 'free/reduced', 'none'])
print(reg.predict(np.array([converted2])))


[[1. 1. 1. 4. 0. 0.]
 [1. 1. 2. 0. 0. 1.]
 [1. 1. 1. 5. 0. 0.]
 ...
 [1. 1. 2. 2. 1. 1.]
 [1. 1. 3. 0. 0. 1.]
 [1. 1. 3. 0. 1. 0.]]
[1, 1, 1, 4, 1, 0]
[59.64825551]
[52.58197253]
